Exemplo 1 - Cravando num dataset

In [ ]:
import h5py
import numpy as np
# ---------------------------
# 1) Definição dos tipos
# ---------------------------
# Strings de tamanho fixo
t_rua    = h5py.string_dtype(encoding='ascii', length=64)
t_cidade = h5py.string_dtype(encoding='ascii', length=32)
t_cep    = h5py.string_dtype(encoding='ascii', length=16)
t_nome   = h5py.string_dtype(encoding='ascii', length=64)

# Tipo composto Endereco
t_endereco = np.dtype([
    ("rua",    t_rua),
    ("cidade", t_cidade),
    ("cep",    t_cep),
])

# Tipo composto Pessoa com array fixo de 3 endereços
t_pessoa = np.dtype([
    ("nome", t_nome),
    ("idade", np.int32),
    ("enderecos", t_endereco, (3,)),  # shape (3,) de t_endereco
])

# ---------------------------
# 2) Dados de exemplo
# ---------------------------
dados = np.zeros(2, dtype=t_pessoa)

dados[0]["nome"] = "Marcos Tavares"
dados[0]["idade"] = 55
dados[0]["enderecos"][0] = ("Rua A, 123", "Rio de Janeiro", "20000-000")
dados[0]["enderecos"][1] = ("Av. Brasil, 500", "Niteroi", "24000-000")
dados[0]["enderecos"][2] = ("Rua das Flores, 10", "Petropolis", "25600-000")

dados[1]["nome"] = "Bruno Lima"
dados[1]["idade"] = 41
dados[1]["enderecos"][0] = ("Rua X, 1", "Sao Paulo", "01000-000")
dados[1]["enderecos"][1] = ("Av. Paulista, 1000", "Sao Paulo", "01310-100")
dados[1]["enderecos"][2] = ("Rua Y, 99", "Campinas", "13000-000")

# ---------------------------
# 3) Criação do arquivo HDF5 e dataset
# ---------------------------
file_path = "pessoas.h5"
with h5py.File(file_path, "w") as f:
    dset = f.create_dataset("pessoas", data=dados, dtype=t_pessoa)

print(f"Arquivo HDF5 criado: {file_path}")

Arquivo HDF5 criado: pessoas.h5
